In [1]:
import gpytorch
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torchvision import transforms

gpytorch.functions.use_toeplitz = False

In [2]:
class FeatureExtractor(nn.Sequential):
    
    def __init__(self):
        super(FeatureExtractor, self).__init__(nn.Conv2d(1, 32, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(32),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2),
                                 nn.Conv2d(32, 64, kernel_size=5, padding=2),
                                 nn.BatchNorm2d(64),
                                 nn.ReLU(),
                                 nn.MaxPool2d(2, 2))
        
class Bottleneck(nn.Sequential):
    
    def __init__(self):
        super(Bottleneck, self).__init__(nn.Linear(64*7*7, 128),
                                         nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128, 128),
                                 nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Linear(128,2),
                                 nn.BatchNorm1d(2))

class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        self.feature_extractor = FeatureExtractor()
        self.bottleneck = Bottleneck()
        self.final_layer = nn.Sequential(
                                 nn.ReLU(),
                                 nn.Linear(2,10))
    
    def forward(self, x):
        features = self.feature_extractor(x)
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        classification = self.final_layer(bottlenecked_features)
        return classification
        

In [3]:
train_mnist = torchvision.datasets.MNIST('/tmp', train=True,
                                         download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))                                              
test_mnist = torchvision.datasets.MNIST('/tmp', train=False,
                                        download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

In [4]:
train_data_loader = torch.utils.data.DataLoader(train_mnist, shuffle=True, pin_memory=True, batch_size=256)

In [5]:
criterion = nn.CrossEntropyLoss().cuda()

In [24]:
model = LeNet().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [25]:
num_epochs = 10
for i in range(num_epochs):
    for x, y in train_data_loader:
        x = Variable(x.cuda())
        y = Variable(y.cuda())
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
    print("Loss: ", loss.data[0])
    

('Loss: ', 1.3508588075637817)
('Loss: ', 1.2423255443572998)
('Loss: ', 0.6224457621574402)
('Loss: ', 0.7861056923866272)
('Loss: ', 0.9659609198570251)
('Loss: ', 0.5116308331489563)
('Loss: ', 0.37412309646606445)
('Loss: ', 0.7509298324584961)
('Loss: ', 0.3614439070224762)
('Loss: ', 0.5085051655769348)


In [57]:
from gpytorch.kernels import RBFKernel, GridInterpolationKernel

class DeepKernel(gpytorch.Module):
    def __init__(self, model):
        super(DeepKernel, self).__init__()
        self.feature_extractor = model.feature_extractor
        self.bottleneck = model.bottleneck
        self.gp_layer = GPLayer()
        
    def forward(self, x):
        print('starting forward')
        features = self.feature_extractor(x)
        bottlenecked_features = self.bottleneck(features.view(-1, 64 * 7 * 7))
        print('gp layer')
        bottlenecked_features *= 0.01
        gp_output = self.gp_layer(bottlenecked_features)
        print('done')
        return gp_output
    
    
class LatentFunction(gpytorch.GridInducingPointModule):
    def __init__(self):
        super(LatentFunction, self).__init__(grid_size=100, grid_bounds=[(-7, 12)])
        cov_module = RBFKernel()
        cov_module.initialize(log_lengthscale=2)
        self.cov_module = cov_module
        
    def forward(self, x):
        mean = Variable(x.data.new(len(x)).zero_())
        covar = self.cov_module(x)
        print(covar.evaluate())
        return gpytorch.random_variables.GaussianRandomVariable(mean, covar)

    
class GPLayer(gpytorch.GPModel):
    def __init__(self, n_dims=2):
        super(GPLayer, self).__init__(gpytorch.likelihoods.BernoulliLikelihood())
        self.latent_functions = [LatentFunction() for i in xrange(n_dims)]
        for i, latent_function in enumerate(self.latent_functions):
            self.add_module('latent_function_%d' % i, latent_function)
    
    def forward(self, x):
        outputs = [latent_function(x[:, i].contiguous()) for i, latent_function in enumerate(self.latent_functions)]
        res = sum(outputs)
        return res
    

In [58]:
deep_kernel = DeepKernel(model).cuda()

In [59]:
gp_data_loader = torch.utils.data.DataLoader(train_mnist, batch_size=128, pin_memory=True)

In [60]:
# Find optimal model hyperparameters
deep_kernel.train()
optimizer = torch.optim.Adam(deep_kernel.gp_layer.parameters(), lr=0.01)
optimizer.n_iter = 0
for i in range(5):
    for j, (train_x_batch, train_y_batch) in enumerate(gp_data_loader):
        train_x_batch = Variable(train_x_batch).cuda()
        train_y_batch = Variable(train_y_batch.fmod(2)).cuda()
        optimizer.zero_grad()
        output = deep_kernel(train_x_batch)
        loss = -deep_kernel.gp_layer.marginal_log_likelihood(output, train_y_batch.float())
        loss.backward()
        optimizer.n_iter += 1
        print('Iter %d/200 - Loss: %.3f' % (
            i + 1, loss.data[0],
        ))
        optimizer.step()
    
# Set back to eval mode
model.eval()
None

starting forward
gp layer
Variable containing:
 1.0000e+00  9.9482e-01  9.7945e-01  ...   6.1783e-22  2.2455e-22  8.0770e-23
 9.9482e-01  1.0000e+00  9.9482e-01  ...   1.6824e-21  6.1784e-22  2.2455e-22
 9.7945e-01  9.9482e-01  1.0000e+00  ...   4.5338e-21  1.6824e-21  6.1783e-22
                ...                   ⋱                   ...                
 6.1783e-22  1.6824e-21  4.5338e-21  ...   1.0000e+00  9.9482e-01  9.7945e-01
 2.2455e-22  6.1784e-22  1.6824e-21  ...   9.9482e-01  1.0000e+00  9.9483e-01
 8.0770e-23  2.2455e-22  6.1783e-22  ...   9.7945e-01  9.9483e-01  1.0000e+00
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000e+00  9.9482e-01  9.7945e-01  ...   6.1783e-22  2.2455e-22  8.0770e-23
 9.9482e-01  1.0000e+00  9.9482e-01  ...   1.6824e-21  6.1784e-22  2.2455e-22
 9.7945e-01  9.9482e-01  1.0000e+00  ...   4.5338e-21  1.6824e-21  6.1783e-22
                ...                   ⋱                   ...                
 6.1783e-22  1.6824e-21 

Iter 1/200 - Loss: 130584.828
starting forward
gp layer
Variable containing:
 1.0000e+00  9.9446e-01  9.7804e-01  ...   2.0830e-23  7.0572e-24  2.3645e-24
 9.9446e-01  1.0000e+00  9.9446e-01  ...   6.0805e-23  2.0830e-23  7.0571e-24
 9.7804e-01  9.9446e-01  1.0000e+00  ...   1.7553e-22  6.0806e-23  2.0830e-23
                ...                   ⋱                   ...                
 2.0830e-23  6.0805e-23  1.7553e-22  ...   1.0000e+00  9.9446e-01  9.7804e-01
 7.0572e-24  2.0830e-23  6.0806e-23  ...   9.9446e-01  1.0000e+00  9.9447e-01
 2.3645e-24  7.0571e-24  2.0830e-23  ...   9.7804e-01  9.9447e-01  1.0000e+00
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000e+00  9.9447e-01  9.7806e-01  ...   2.1703e-23  7.3592e-24  2.4678e-24
 9.9447e-01  1.0000e+00  9.9447e-01  ...   6.3300e-23  2.1703e-23  7.3591e-24
 9.7806e-01  9.9447e-01  1.0000e+00  ...   1.8259e-22  6.3300e-23  2.1703e-23
                ...                   ⋱                   ...           

Iter 1/200 - Loss: 39107.590
starting forward
gp layer
Variable containing:
 1.0000e+00  9.9417e-01  9.7688e-01  ...   1.2846e-24  4.1081e-25  1.2984e-25
 9.9417e-01  1.0000e+00  9.9417e-01  ...   3.9704e-24  1.2846e-24  4.1080e-25
 9.7688e-01  9.9417e-01  1.0000e+00  ...   1.2129e-23  3.9705e-24  1.2846e-24
                ...                   ⋱                   ...                
 1.2846e-24  3.9704e-24  1.2129e-23  ...   1.0000e+00  9.9417e-01  9.7688e-01
 4.1081e-25  1.2846e-24  3.9705e-24  ...   9.9417e-01  1.0000e+00  9.9417e-01
 1.2984e-25  4.1080e-25  1.2846e-24  ...   9.7688e-01  9.9417e-01  1.0000e+00
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000e+00  9.9418e-01  9.7694e-01  ...   1.4734e-24  4.7251e-25  1.4977e-25
 9.9418e-01  1.0000e+00  9.9418e-01  ...   4.5411e-24  1.4734e-24  4.7251e-25
 9.7694e-01  9.9418e-01  1.0000e+00  ...   1.3834e-23  4.5411e-24  1.4734e-24
                ...                   ⋱                   ...            

Iter 1/200 - Loss: 17058.877
starting forward
gp layer
Variable containing:
 1.0000e+00  9.9396e-01  9.7606e-01  ...   1.7747e-25  5.4473e-26  1.6518e-26
 9.9396e-01  1.0000e+00  9.9396e-01  ...   5.7125e-25  1.7747e-25  5.4472e-26
 9.7606e-01  9.9396e-01  1.0000e+00  ...   1.8166e-24  5.7126e-25  1.7747e-25
                ...                   ⋱                   ...                
 1.7747e-25  5.7125e-25  1.8166e-24  ...   1.0000e+00  9.9396e-01  9.7606e-01
 5.4473e-26  1.7747e-25  5.7126e-25  ...   9.9396e-01  1.0000e+00  9.9396e-01
 1.6518e-26  5.4472e-26  1.7747e-25  ...   9.7606e-01  9.9396e-01  1.0000e+00
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000e+00  9.9397e-01  9.7609e-01  ...   1.9118e-25  5.8769e-26  1.7849e-26
 9.9397e-01  1.0000e+00  9.9397e-01  ...   6.1443e-25  1.9118e-25  5.8769e-26
 9.7609e-01  9.9397e-01  1.0000e+00  ...   1.9510e-24  6.1443e-25  1.9118e-25
                ...                   ⋱                   ...            

Iter 1/200 - Loss: 17001.662
starting forward
gp layer
Variable containing:
 1.0000e+00  9.9379e-01  9.7539e-01  ...   3.4757e-26  1.0314e-26  3.0224e-27
 9.9379e-01  1.0000e+00  9.9379e-01  ...   1.1568e-25  3.4757e-26  1.0313e-26
 9.7539e-01  9.9379e-01  1.0000e+00  ...   3.8025e-25  1.1568e-25  3.4757e-26
                ...                   ⋱                   ...                
 3.4757e-26  1.1568e-25  3.8025e-25  ...   1.0000e+00  9.9379e-01  9.7539e-01
 1.0314e-26  3.4757e-26  1.1568e-25  ...   9.9379e-01  1.0000e+00  9.9379e-01
 3.0224e-27  1.0313e-26  3.4757e-26  ...   9.7539e-01  9.9379e-01  1.0000e+00
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000e+00  9.9381e-01  9.7547e-01  ...   4.2505e-26  1.2665e-26  3.7274e-27
 9.9381e-01  1.0000e+00  9.9381e-01  ...   1.4089e-25  4.2505e-26  1.2665e-26
 9.7547e-01  9.9381e-01  1.0000e+00  ...   4.6121e-25  1.4089e-25  4.2505e-26
                ...                   ⋱                   ...            

...

Iter 5/200 - Loss: 95.744
starting forward
gp layer
Variable containing:
 1.0000  0.8846  0.6123  ...   0.0000  0.0000  0.0000
 0.8846  1.0000  0.8846  ...   0.0000  0.0000  0.0000
 0.6123  0.8846  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8846  0.6123
 0.0000  0.0000  0.0000  ...   0.8846  1.0000  0.8847
 0.0000  0.0000  0.0000  ...   0.6123  0.8847  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000  0.8830  0.6080  ...   0.0000  0.0000  0.0000
 0.8830  1.0000  0.8830  ...   0.0000  0.0000  0.0000
 0.6080  0.8830  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8830  0.6080
 0.0000  0.0000  0.0000  ...   0.8830  1.0000  0.8831
 0.0000  0.0000  0.0000  ...   0.6080  0.8831  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

done
Iter 5/200 - Loss: 141.956
starting forward
g

Iter 5/200 - Loss: 120.694
starting forward
gp layer
Variable containing:
 1.0000  0.8842  0.6113  ...   0.0000  0.0000  0.0000
 0.8842  1.0000  0.8842  ...   0.0000  0.0000  0.0000
 0.6113  0.8842  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8842  0.6113
 0.0000  0.0000  0.0000  ...   0.8842  1.0000  0.8843
 0.0000  0.0000  0.0000  ...   0.6113  0.8843  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000  0.8827  0.6071  ...   0.0000  0.0000  0.0000
 0.8827  1.0000  0.8827  ...   0.0000  0.0000  0.0000
 0.6071  0.8827  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8827  0.6071
 0.0000  0.0000  0.0000  ...   0.8827  1.0000  0.8828
 0.0000  0.0000  0.0000  ...   0.6071  0.8828  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

done
Iter 5/200 - Loss: 179.890
starting forward


Iter 5/200 - Loss: 68.817
starting forward
gp layer
Variable containing:
 1.0000  0.8838  0.6102  ...   0.0000  0.0000  0.0000
 0.8838  1.0000  0.8838  ...   0.0000  0.0000  0.0000
 0.6102  0.8838  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8838  0.6102
 0.0000  0.0000  0.0000  ...   0.8838  1.0000  0.8839
 0.0000  0.0000  0.0000  ...   0.6102  0.8839  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000  0.8824  0.6064  ...   0.0000  0.0000  0.0000
 0.8824  1.0000  0.8824  ...   0.0000  0.0000  0.0000
 0.6064  0.8824  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8824  0.6064
 0.0000  0.0000  0.0000  ...   0.8824  1.0000  0.8825
 0.0000  0.0000  0.0000  ...   0.6064  0.8825  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

done
Iter 5/200 - Loss: 166.826
starting forward
g

Iter 5/200 - Loss: 129.473
starting forward
gp layer
Variable containing:
 1.0000  0.8835  0.6093  ...   0.0000  0.0000  0.0000
 0.8835  1.0000  0.8835  ...   0.0000  0.0000  0.0000
 0.6093  0.8835  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8835  0.6093
 0.0000  0.0000  0.0000  ...   0.8835  1.0000  0.8836
 0.0000  0.0000  0.0000  ...   0.6093  0.8836  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000  0.8821  0.6056  ...   0.0000  0.0000  0.0000
 0.8821  1.0000  0.8821  ...   0.0000  0.0000  0.0000
 0.6056  0.8821  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8821  0.6056
 0.0000  0.0000  0.0000  ...   0.8821  1.0000  0.8822
 0.0000  0.0000  0.0000  ...   0.6056  0.8822  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

done
Iter 5/200 - Loss: 181.023
starting forward


Iter 5/200 - Loss: 148.055
starting forward
gp layer
Variable containing:
 1.0000  0.8831  0.6083  ...   0.0000  0.0000  0.0000
 0.8831  1.0000  0.8831  ...   0.0000  0.0000  0.0000
 0.6083  0.8831  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8831  0.6083
 0.0000  0.0000  0.0000  ...   0.8831  1.0000  0.8832
 0.0000  0.0000  0.0000  ...   0.6083  0.8832  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

Variable containing:
 1.0000  0.8818  0.6047  ...   0.0000  0.0000  0.0000
 0.8818  1.0000  0.8818  ...   0.0000  0.0000  0.0000
 0.6047  0.8818  1.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   1.0000  0.8818  0.6047
 0.0000  0.0000  0.0000  ...   0.8818  1.0000  0.8819
 0.0000  0.0000  0.0000  ...   0.6047  0.8819  1.0000
[torch.cuda.FloatTensor of size 100x100 (GPU 0)]

done
Iter 5/200 - Loss: 145.243
starting forward


In [41]:
deep_kernel.gp_layer.condition(Variable(torch.randn(2, 2)).cuda(), Variable(torch.zeros(2)).cuda())

GPLayer (
  (likelihood): BernoulliLikelihood (
  )
  (latent_function_0): LatentFunction (
    (cov_module): GridInterpolationKernel (
      (base_kernel_module): RBFKernel (
      )
    )
  )
  (latent_function_1): LatentFunction (
    (cov_module): GridInterpolationKernel (
      (base_kernel_module): RBFKernel (
      )
    )
  )
)

In [48]:
deep_kernel.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

for test_batch_x, test_batch_y in test_data_loader:
    predictions = deep_kernel(Variable(test_batch_x).cuda()).probability.round()
    print(predictions)
    test_batch_y = Variable(test_batch_y.fmod(2)).cuda().float()
    print(torch.eq(predictions, test_batch_y).float().mean().data[0])


starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.55859375
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.546875
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 

starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.50390625
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.484375
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 

starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.4921875
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1


...

done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.51171875
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 

starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.42578125
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.53125
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.50390625
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 

gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.cuda.FloatTensor of size 256 (GPU 0)]

0.46484375
starting forward
gp layer
done
Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 

In [47]:
from torch.nn import functional as F

model.eval()
test_data_loader = torch.utils.data.DataLoader(test_mnist, shuffle=False, pin_memory=True, batch_size=256)

for test_batch_x, test_batch_y in test_data_loader:
    _, predictions = model(Variable(test_batch_x).cuda()).max(dim=1)
    predictions = predictions.fmod(2)
    #print(predictions)
    test_batch_y = Variable(test_batch_y).cuda().fmod(2)
    #print(test_batch_y)
    print(torch.eq(predictions, test_batch_y).float().mean().data[0])


0.99609375
0.98046875
0.9609375
0.96484375
0.953125
0.9765625
0.96875
0.984375
0.984375
0.97265625
0.98828125
0.96875
0.984375
0.984375
0.96875
0.9765625
0.984375
0.9765625
0.94921875
0.97265625
0.98828125
0.98828125
0.98828125
0.98828125
0.9921875
0.96484375
0.984375
0.9921875
0.9921875
0.9921875
0.9921875
0.984375
0.96875
0.984375
1.0
0.98828125
0.98828125
0.9765625
0.98046875
0.9375
